## Create a simple solar system model

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from collections import namedtuple

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/matplotlib/font_manager.py:229: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


### Define a planet class

In [2]:
class planet():
    "A planet in our solar system"
    def __init__(self,semimajor,eccentricity):
        self.x     = np.zeros(2)  #x and y position
        self.v     = np.zeros(2)  #x and y velocity
        self.a_g   = np.zeros(2)  #x and y acceleration
        self.t     = 0.0          #current time
        self.dt    = 0.0          #current timestep
        self.a     = semimajor    #semimajor axis of the orbit
        self.e     = eccentricity #eccentricity of the orbit
        self.istep = 0            #current integer timestep1
        self.name  = ""           #name for the planet

### Define a dictionary with some constants

In [10]:
solar_system = { "M_sun":1.0, "G":39.4784176043574320} #G = (4*np.pi)**2

### Define some functions for setting circular velocity and acceleration

In [11]:
def solar_circular_velocity(p):
    
    G = solar_system["G"]
    M = solar_system["M_sun"]
    r = (p.x[0]**2 + p.x[1]**2)**0.5
    
    #return the circular velocity
    return(G*M/r)**0.5

In [12]:
def solar_gravitational_acceleration(p):
    
    G = solar_system["G"]
    M = solar_system["M_sun"]
    r = (p.x[0]**2 + p.x[1]**2)**0.5
    
    #acceleration in AU/(yr^2)
    a_grav = -1.0*G*M/r**2
    
    #find the angle at this position
    if(p.x[0]==0.0):
        if(p.x[1]>0.0):
            theta = 0.5*np.pi
        else:
            theta = 1.5*np.pi
    else:
        theta = np.atan(p.x[1],p.x[0]) #arctan p.x[1]/p.x[0]
        
    #set the x and y components of the velocity
    #p.a_g[0] = a_grav * np.cos(theta)
    #p.a_g[1] = a_grav * np.sin(theta)
    return a_grav*np.cos(theta), a_grav*np.sin(theta)

### Compute the timestep

In [13]:
def calc_dt(p):
    
    #integration tolerance
    ETA_TIME_STEP = 0.0004
    
    #compute timestep
    eta = ETA_TIME_STEP
    v = (p.v[0]**2 + p.v[1]**2)**0.5
    a = (p.a_g[0]**2 + p.a_g[1]**2)**0.5
    dt = eta * fp.min(1./np.fabs(v),1./np.fabs(a)**0.5)
    
    return dt

### Define the initial conditions

In [14]:
def SetPlanet(p,i):
    
    AU_in_km = 1.495979e+8 #an AU in km
    
    #circular velocity
    v_c = 0.0    #circular velocity in AU/yr
    v_e = 0.0    #velocity at perihelion in AU/yr
    
    #planety by planet initial conditions
    
    #Mercury
    if(i==0):
        #semi-major axis in AU
        p.a = 57909227.0 #AU_in_km
        
        #eccentricity
        p.e = 0.20563593
        
        #name
        p.name = "Mercury"
        
    #Venus
    elif(i==1):
        #semi-major axis in AU
        p.a = 108209475.0 #AU_in_km
        
        #eccentricity
        p.e = 0.00677672
        
        #name
        p.name = "Venus"
        
    #Earth
    elif(i==2):
        #semi-major axis in AU
        p.a = 1.0
        
        #eccentricity
        p.e = 0.01671123
        
        #name
        p.name = "Earth"
        
    #set remaining properties
    p.t    = 0.0
    p.x[0] = p.a*(1.0-p.e)
    p.x[1] = 0.0
    
    #get equiv circular velocity
    v_c = solar_circular_velocity(p)
    
    #velocity at perihelion
    v_e = v_c*(1 + p.e)**0.5
    
    #set velocity
    p.v[0] = 0.0   #no x velocity at perihelion
    p.v[1] = v_e   #y velocity at perihelion (CCW)
    
    #calculate gravitational acceleration from sun
    p.a_g = solar_gravitational_acceleration(p)
    
    #set timestep
    p.dt = calc_dt(p)

### Write leapfrog integrator

In [15]:
def x_first_step(x_i,v_i,a_i,dt):
    #x_1/2 = x_0 + 1/2 v_0 Delta_t + 1/4 a_0 Delta t^2
    return x_i + 0.5*v_i*dt + 0.25*a_i*dt**2

In [20]:
def v_full_step(v_i,a_ipoh,dt):
    #v_i+1 = v_i + a_i+1/2 Delta t
    #ipoh means i plus one half
    return v_i + a_ipoh*dt;

In [21]:
def x_full_step(x_ipoh,v_ip1,a_ipoh,dt):
    #x_3/2 = x_1/2 + v_i+1 Delta t
    return x_ipoh + v_ip1*dt;

### Write a function to save the data to file

In [22]:
def save_solar_system(p,n_planets,t,dt,istep,ndim):
    
    #loop over the number of planets
    for i in range(n_planets):
        
        #define a filename
        fname = "planet.%s.txt" % p[i].name
        
        if(istep==0):
            #create the file on the first timestep
            fp = open(fname,"w")
        else:
            #append the file on subsequent timesteps
            fp = open(fname,"a")
            
        #compute the drifted properties of the planet
        v_drift = np.zeros(n.dim)
        
        for k in range(ndim):
            v_drift[k] = p[i].v[k] + 0.5*p[i].a_g[k]*p[i].dt
            
        #write the data to file
        s = "%6d\t%6.5f\t%6.5f\t%6d\t%6.5f\t%6.5f\t% 6.5f\t% 6.5f\t% 6.5f\t% 6.5f\t% 6.5f\t% 6.5f\n" % \
            (istep,t,dt,p[i].istep,p[i].t,p[i].dt,p[i].x[0],p[i].x[1],v_drift[0],v_drift[1],     \
             p[i].a_g[0],p[i].a_g[1])
        fp.write(s)
        
        
        #close the file
        fp.close()

### Write a function to evolve the solar system

In [23]:
def evolve_solar_system(p,n_planets,t_max):
    
    #number of spatial dimensions
    ndim = 2
    
    #define the first timestep
    dt = 0.55/365.25
    
    #define the starting time
    t = 0.0
    
    #define the starting timestep
    istep = 0
    
    #save the initial conditions
    save_solar_system(p,n_planets,t,dt,istep,ndim)
    
    #begin a loop over the global timescale
    while(t<t_max):
        
        #check to see if the next step exceeds the max time;
        #if so, take a smaller step
        if(t+dt>t_max):
            dt = t_max - t #limit the step to align with t_max
            
        #evolve each planet
        for i in range(n_planets):
            
            while(p[i].t<t+dt):
                
                #special case for istep==0
                if(p[i].istep==0):
                    
                    #take the first step according to a verlet scheme
                    for k in range(ndim):
                        p[i].x[k] = x_first_step(p[i].x[k],p[i].v[k],p[i].a_g[k],p[i].dt)
                        
                    #update the acceleration
                    p[i].a_g = solar_gravitational_acceleration(p[i])
                    
                    #update the time by 1/2dt
                    p[i].t += 0.5*p[i].dt
                    
                    #update the timestep
                    p[i].dt = calc_dt(p[i])
                    
                #continue with a normal step
                
                #limit to align with the global timestep
                if(p[i].t + p[i].dt > t+dt):
                    p[i].dt = t+dt-p[i].t
                    ####left off at slide 26####